In [1]:
import pandas as pd
import geopy.distance
import problem
from merge_transformer import MergeTransformer
from create_db import create_db
import os

In [2]:
from create_db import create_db

database = create_db()

           State     2018     2017     2016     2015     2014    2013    2012  \
0           D.C.  160,472  159,227  159,395  159,497  163,274  166,87  168,03   
1  Massachusetts   65,545   64,507    62,51   61,882   62,456  61,769  60,808   
2       New York   64,579   64,093    63,42   62,444   62,841  61,185  61,267   
3    Connecticut   64,511   63,747   62,236    62,55   63,502  63,638  64,906   
4         Alaska   63,971   67,705   67,411     69,7   73,478  70,573  69,564   

      2011  
0  166,178  
1   59,178  
2   59,481  
3   65,574  
4   72,204  




        State       Date UnemploymentRate  year  month  day  weekday  week  \
0     Alabama 2011-01-01              9.3  2011      1    1        5    52   
1      Alaska 2011-01-01              7.7  2011      1    1        5    52   
2     Arizona 2011-01-01              9.6  2011      1    1        5    52   
3    Arkansas 2011-01-01              7.8  2011      1    1        5    52   
4  California 2011-01-01             12.4

In [3]:
X_train, y_train = problem.get_train_data('..')
X_train.loc[:, "DateOfDeparture"] = pd.to_datetime(X_train['DateOfDeparture'])

X_test, y_test = problem.get_test_data('..')
X_test.loc[:, "DateOfDeparture"] = pd.to_datetime(X_test['DateOfDeparture'])

In [4]:
def merge_dfs(X):
    
    database = create_db()
        
    merge_transform = MergeTransformer(X_ext=database['Date'], how='left', on=['DateOfDeparture'])
    X = merge_transform.fit_transform(X)
        
    merge_transform = MergeTransformer(
        X_ext=database['Airport'], 
        cols_to_rename={'iata': 'Departure',
                        'latitude_deg': 'latitude_departure',
                        'longitude_deg': 'longitude_departure',
                        'state': 'state_departure',
                        'capacity': 'capacity_departure'},
        how='inner', on=['Departure'])
    X = merge_transform.fit_transform(X)

    merge_transform = MergeTransformer(
        X_ext=database['Airport'], 
        cols_to_rename={'iata': 'Arrival', 
                        'latitude_deg': 'latitude_arrival', 
                        'longitude_deg': 'longitude_arrival',
                        'state': 'state_arrival',
                        'capacity': 'capacity_arrival'},
        how='inner', on=['Arrival'])

    X = merge_transform.fit_transform(X)
    
    X['distance'] = X.apply(lambda x: geopy.distance.geodesic(
        (x.latitude_departure, x.longitude_departure), 
        (x.latitude_arrival, x.longitude_arrival)).km, axis=1)
    
    merge_transform = MergeTransformer(
        X_ext=database['Weather'], 
        cols_to_rename={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'}, 
        how='left',
        on=['DateOfDeparture', 'Arrival'])

    X = merge_transform.fit_transform(X)

    merge_transform = MergeTransformer(
        X_ext=database['StateFeatures'],
        cols_to_keep=['DateOfDeparture', 'state', 'UnemploymentRate', 'holidays'],
        cols_to_rename={'state': 'state_departure'}, 
        how='left',
        on=['DateOfDeparture', 'state_departure'])
    
#     print(database['StateFeatures'].head())

    X = merge_transform.fit_transform(X)
    
    return X

In [ ]:
X_train = merge_dfs(X_train)

In [ ]:
X_test = merge_dfs(X_test)

In [ ]:
X_train.to_csv('../data/X_train.csv', index=False)
X_test.to_csv('../data/X_test.csv', index=False)

In [ ]:
y_train_pd = pd.Series(y_train)
y_test_pd = pd.Series(y_test)

y_train_pd.to_csv('../data/y_train.csv', index=False)
y_test_pd.to_csv('../data/y_test.csv', index=False)